In [9]:
#CSE - Sudoku Solver using Crook's Algorithm
#COMPRA, IMEE
#DALUMPINES, SOPHIA
#DELOS REYES, CHRISTINE
#FLOR, LEA
#BSCS 3AB

import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
import openpyxl
import pandas as pd

def gridScan(candidates, soln): #scan each 3x3 box for possible candidates
    for i in [1, 4, 7]:
        for j in [1, 4, 7]:
            given = set(
                [soln[i_range - 1][j_range - 1] for j_range in range(j, j + 3) for i_range in range(i, i + 3)])
            for k in puzzleRange(i):
                for l in puzzleRange(j):
                    candidates[k, l] = [b for b in candidates[k, l] if b not in given]

            nom = [x for b in [value for key, value in candidates.items()
                                            if key[0] in puzzleRange(i) and key[1] in puzzleRange(j) and len(value) > 0]
                                for x in b]
            unique = [x for x in nom if nom.count(x) == 1]
            if len(unique) > 0:
                for k in unique:
                    for key, value in {key: value for key, value in candidates.items()
                                       if key[0] in puzzleRange(i) and key[1] in puzzleRange(j) and len(value) > 0}.items():
                        if k in value:
                            soln[key[0] - 1][key[1] - 1] = k
                            candidates[key] = []
    return 0

def columnScan(candidates, soln): #scan each column for possible candidates
    for j in range(1, 10):
        given = [x[j - 1] for x in soln]
        for i in range(1, 10):
            candidates[i, j] = [x for x in candidates[i, j] if x not in given]

        nom = [x for y in [value for key, value in candidates.items()
                                        if key[1] == j and len(value) > 0] for x in y]
        unique = [x for x in nom if nom.count(x) == 1]
        if len(unique) > 0:
            for x in unique:
                for key, value in {key: value for key, value in candidates.items() if
                                   key[1] == j and len(value) > 0}.items():
                    if x in value:
                        soln[key[0] - 1][key[1] - 1] = x
                        candidates[key] = []
    return 0

def rowScan(candidates, soln): #scan each row for possible candidates
    for i in range(1, 10):
        given = soln[i - 1]

        for j in range(1, 10):
            candidates[i, j] = [x for x in candidates[i, j] if x not in given]

        nom = [x for y in [value for key, value in candidates.items()
                                        if key[0] == i and len(value) > 0] for x in y]
        unique = [x for x in nom if nom.count(x) == 1]
        if len(unique) > 0:
            for x in unique:
                for key, value in {key: value for key, value in candidates.items() if
                                   key[0] == i and len(value) > 0}.items():
                    if x in value:
                        soln[key[0] - 1][key[1] - 1] = x
                        candidates[key] = []
    return 0

solutions=[]

def solvePuzzle(candidates, soln): #initial solving
    rowScan(candidates, soln)
    columnScan(candidates, soln)
    gridScan(candidates, soln)
    solutions.append(soln)
    return solutions

def scanCandidates(candidates, soln): #scan for candidates
    for key, value in candidates.items():
        if len(value) == 1:
            soln[key[0] - 1][key[1] - 1] = value[0]
            candidates[key] = []
    return 0

def fundamentalSol(candidates, soln): #perform basic solving
    while True:
        prevSolution = copy.deepcopy(soln)
        rowScan(candidates, soln)
        columnScan(candidates, soln)
        gridScan(candidates, soln)
        scanCandidates(candidates, soln)
        if soln == prevSolution:
            break

def crookAlgo(markup, candidates, soln): #apply crook's algorithm
    try:
        mini = min((len(v)) for _, v in markup.items())
        maxi = max((len(v)) for _, v in markup.items())
    except ValueError:
        return 0
    for i in reversed(range(mini, maxi + 1)):
        for key, value in {key: value for key, value in markup.items() if len(value) == i}.items():
            n_subset = 0
            key_match = set()
            for key_1, value_1 in markup.items():
                if len(value) < len(value_1):
                    continue
                else:
                    if set(value_1).issubset(set(value)):
                        key_match.add(key_1)
                        n_subset += 1
                if n_subset == len(value):
                    for key_2, value_2 in {key: value for key, value in markup.items() if
                                           key not in key_match}.items():
                        candidates[key_2] = [x for x in value_2 if x not in value]
                        fundamentalSol(candidates, soln)
    return 0

def crook_gridScan(candidates, soln): #scan each 3x3 box using crook's algorithm
    for i in [1, 4, 7]:
        for j in [1, 4, 7]:
            markups = {key: value for key, value in candidates.items() 
                       if key[0] in puzzleRange(i) and key[1] in puzzleRange(j) and len(value) > 0}
            crookAlgo(markups, candidates, soln)
    return 0

def crook_columnScan(candidates, soln): #scan each column using crook's algorithm
    for j in range(1, 10):
        markups = {key: value for key, value in candidates.items() 
                   if key[1] == j and len(value) > 0}
        crookAlgo(markups, candidates, soln)
    return 0

def crook_rowScan(candidates, soln): #scan each row using crook's algorithm
    for i in range(1, 10):
        markups = {key: value for key, value in candidates.items() 
                   if key[0] == i and len(value) > 0}
        crookAlgo(markups, candidates, soln)
    return 0

def crookSol(candidates, soln): #solve using crook's algorithm until possible
    while True:
        prevSolution = copy.deepcopy(soln)
        crook_rowScan(candidates, soln)
        crook_columnScan(candidates, soln)
        crook_gridScan(candidates, soln)
        if soln == prevSolution:
            break
    return 0

def gridElim(candidates): #eliminate candidates
    for i in [1, 4, 7]:
        for j in [1, 4, 7]:
            nom = set([x for b in
                                    [value for key, value in candidates.items()
                                     if key[0] in puzzleRange(i) and key[1] in puzzleRange(j) and len(value) > 0]
                                    for x in b])

            for x in nom:
                available_cell = [key for key, value in candidates.items()
                                  if x in value if key[0] in puzzleRange(i) and key[1] in puzzleRange(j)]
                if len(set([x[0] for x in available_cell])) == 1:
                    for key in [key for key, value in candidates.items() if
                                key[0] == available_cell[0][0] and key not in available_cell]:
                        candidates[key] = [y for y in candidates[key] if y != x]
                if len(set([x[1] for x in available_cell])) == 1:
                    for key in [key for key, value in candidates.items() if
                                key[1] == available_cell[0][1] and key not in available_cell]:
                        candidates[key] = [y for y in candidates[key] if y != x]
    return 0

def puzzleRange(number): #range of numbers within the puzzle
    if number in (1, 2, 3):
        return [1, 2, 3]
    elif number in (4, 5, 6):
        return [4, 5, 6]
    elif number in (7, 8, 9):
        return [7, 8, 9]

def readPuzzle(excelFile, puzzleSheet): #read sudoku puzzle in excel file
    file = pd.read_excel(excelFile, sheet_name = puzzleSheet, header = None)
    return file.values.tolist()

In [10]:
import copy

root = tk.Tk()
root.title("Sudoku Solver")
root.geometry("500x650")
root.resizable(False, False)

tit = tk.Label(root, text="Sudoku Solver", height=1, width=55, fg="white", bg='brown', font=("", 12)).grid(row=1)

path_label = tk.Label(root, text="Excel File Path:", font=("", 10), fg="brown")
path_label.grid(row=2, column=0, sticky="W", padx=5, pady=5)

fileField = tk.Entry(root)
fileField.grid(row=2, column=0, padx=5, pady=5)
    
def browse_file(): #browse excel file from computer
    file_path = filedialog.askopenfilename()
    fileField.delete(0, tk.END)
    fileField.insert(0, file_path)
    
def clear(): #clear the input
    fileField.delete(0, tk.END)
    sheet_entry.delete(0, tk.END)
    result_text.configure(state='normal')
    result_text.delete('1.0', tk.END)
    result_text.configure(state='disabled')

def solveSudoku(): #solve and display the solution
    filePath = fileField.get()
    sheet_name = sheet_entry.get()
    puzzle = readPuzzle(filePath, sheet_name)

    solution = copy.deepcopy(puzzle)
    candidate = {}

    for i in range(1, 10):
        for j in range(1, 10):
            candidate[i, j] = list(range(1, 10))

    for i in range(1, 10):
        for j in range(1, 10):
            if solution[i - 1][j - 1] != 0:
                candidate[i, j] = []

    result = "Sudoku puzzle:\n"
    for i in puzzle:
        result += str(i) + "\n"
    result += "\n"

    while True:
        fundamentalSol(candidate, solution)
        result += "Solution before applying preemptive sets:\n"
        for i in solution:
            result += str(i) + "\n"
        result += "\n"

        crookSol(candidate, solution)
        prevSolution = copy.deepcopy(solution)
        gridElim(candidate)
        
        result += "Solution after applying preemptive sets:\n"
        for i in solution:
            result += str(i) + "\n"
        result += "\n"
        
        if solution == prevSolution:
            break
            
    if (
            min([sum(x) for x in zip(*solution)]) == max([sum(x) for x in zip(*solution)])
            and min([sum(x) for x in solution]) == max([sum(x) for x in solution])
            and min([sum(x) for x in zip(*solution)]) == 45
            and min([sum(x) for x in solution]) == 45
    ):
        result += "Final solution:\n"
    else:
        result += "Solution not yet found. Current solution is:\n"
        
    for i in solution:
        result += str(i) + "\n"
       
    result_text.configure(state='normal')
    result_text.delete('1.0', tk.END)
    result_text.insert(tk.END, result)
    result_text.configure(state='disabled')

browse_button = tk.Button(root, text="Browse", command=browse_file, fg="white", bg='brown', font=("", 10))
browse_button.grid(row=2, column=0, padx=10, pady=10, sticky="E")

sheet_label = tk.Label(root, text="Sheet Name:", fg="brown", font=("", 10))
sheet_label.grid(row=3, column=0, sticky="W", padx=5, pady=5)

sheet_entry = tk.Entry(root)
sheet_entry.grid(row=3, column=0, padx=5, pady=5)

solve_button = tk.Button(root, text="Solve", command=solveSudoku, fg="white", bg="brown", padx=35, pady=10, font=("", 10))
solve_button.grid(row=4, column=0, sticky="W")

result_label = tk.Label(root, text="Crook's Solution:", fg="brown", font=("", 12))
result_label.grid(row=5, column=0, padx=5, pady=5)

clear_button = tk.Button(root, text="Clear", command=clear, fg="white", bg="brown", padx=35, pady=10, font=("", 10))
clear_button.grid(row=4, column=0, sticky="E")

result_text = tk.Text(root, height=28, width=60)
result_text.grid(row=6, column=0, columnspan=3, padx=5, pady=5)
result_text.configure(state='disabled')

root.mainloop()